In [1]:
import tensorflow as tf
import json
import numpy as np
from sklearn.model_selection import train_test_split

2023-06-06 13:32:01.383887: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 13:32:01.419892: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 13:32:01.420741: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 13:32:02.410595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_path = '../dataset.json'

# Open the file and load its contents
with open(file_path, 'r') as json_file:
    # Read the file content
    file_content = json_file.read()

    # Split the content by newline to get individual JSON objects
    json_objects = file_content.split('\n')

data = []
# Process each JSON object
for json_object in json_objects:
    if json_object.strip() == '':
        continue  # Skip empty lines
    
    # Parse the JSON object
    data.append(json.loads(json_object))

print(len(data))

X = []
Y = []
Y_flat = []

for i in data:
    X.append(np.array(i['vector']))    
    Y.append(np.array(i['matrix']))
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
Y_train_flat =[i.flatten() for i in Y_train]
Y_test_flat =[i.flatten() for i in Y_test]

1331


# Handmade NN

In [3]:
# Define the neural network architecture
def build_crazy_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(output_dim)
    ])
    return model
def build_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(196)  # Output layer with shape (196,)
    ])
    return model

def build_model_recursive(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Reshape((1, 256)),  # Reshape input to match LSTM requirements
        # tf.keras.layers.LSTM(512, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='relu')),
        tf.keras.layers.LSTM(512),
        tf.keras.layers.Dense(output_dim)
    ])
    return model

# def build_model_convolutional(input_dim):
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
#         tf.keras.layers.Dense(7*7*128, activation='relu'),
#         tf.keras.layers.Reshape((7, 7, 128)),
#         tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2DTranspose(2, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2D(1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Reshape((14, 14, 1))  # Update the output shape to (14, 14, 1)
#     ])
#     return model

In [4]:
# Define the input and output dimensions
input_dim = 128  # Dimensionality of the input vector
output_dim = 196  # Dimensionality of the output vector

In [5]:
# Build the model
model = build_model_recursive(input_dim, output_dim)
model_ann = build_model(input_dim, output_dim)
model_conv = build_model_convolutional(input_dim)

# Compile the model
model.compile(optimizer='adam', loss='mse')
model_ann.compile(optimizer='adam', loss='mse')
model_conv.compile(optimizer='adam', loss='mse')

2023-06-06 13:32:09.039836: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-06 13:32:09.087598: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-06 13:32:09.462638: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gra

In [6]:
model_ann.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=800)

Epoch 1/800
32/32 [==============================] - 1s 10ms/step - loss: 22744204.0000
Epoch 2/800
32/32 [==============================] - 0s 9ms/step - loss: 1294.5846
Epoch 3/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2776
Epoch 4/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2670
Epoch 5/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2666
Epoch 6/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2667
Epoch 7/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2667
Epoch 8/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2662
Epoch 9/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2661
Epoch 10/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2661
Epoch 11/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2660
Epoch 12/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2

32/32 [==============================] - 0s 9ms/step - loss: 1289.2181
Epoch 99/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2167
Epoch 100/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2159
Epoch 101/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2147
Epoch 102/800
32/32 [==============================] - 0s 10ms/step - loss: 1289.2139
Epoch 103/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2124
Epoch 104/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2114
Epoch 105/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2101
Epoch 106/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2094
Epoch 107/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2080
Epoch 108/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.2067
Epoch 109/800
32/32 [==============================] - 0s 9ms/step - loss: 1289

32/32 [==============================] - 0s 9ms/step - loss: 1289.0602
Epoch 195/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.0577
Epoch 196/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.0563
Epoch 197/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.0544
Epoch 198/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.0530
Epoch 199/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.0511
Epoch 200/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.0492
Epoch 201/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.0474
Epoch 202/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.0455
Epoch 203/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.0439
Epoch 204/800
32/32 [==============================] - 0s 9ms/step - loss: 1289.0416
Epoch 205/800
32/32 [==============================] - 0s 9ms/step - loss: 1289

32/32 [==============================] - 0s 9ms/step - loss: 1288.8625
Epoch 291/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.8607
Epoch 292/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.8579
Epoch 293/800
32/32 [==============================] - 0s 10ms/step - loss: 1288.8566
Epoch 294/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.8536
Epoch 295/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.8518
Epoch 296/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.8494
Epoch 297/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.8469
Epoch 298/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.8447
Epoch 299/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.8428
Epoch 300/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.8405
Epoch 301/800
32/32 [==============================] - 0s 9ms/step - loss: 128

32/32 [==============================] - 0s 9ms/step - loss: 1288.6642
Epoch 387/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.6626
Epoch 388/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.6611
Epoch 389/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.6597
Epoch 390/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.6577
Epoch 391/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.6561
Epoch 392/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.6543
Epoch 393/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.6527
Epoch 394/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.6510
Epoch 395/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.6492
Epoch 396/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.6478
Epoch 397/800
32/32 [==============================] - 0s 9ms/step - loss: 1288

32/32 [==============================] - 0s 9ms/step - loss: 1288.4834
Epoch 483/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.4817
Epoch 484/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.4801
Epoch 485/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.4775
Epoch 486/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.4755
Epoch 487/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.4734
Epoch 488/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.4718
Epoch 489/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.4703
Epoch 490/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.4679
Epoch 491/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.4663
Epoch 492/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.4637
Epoch 493/800
32/32 [==============================] - 0s 9ms/step - loss: 1288

32/32 [==============================] - 0s 9ms/step - loss: 1288.2996
Epoch 579/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.2979
Epoch 580/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.2960
Epoch 581/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.2942
Epoch 582/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.2924
Epoch 583/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.2902
Epoch 584/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.2885
Epoch 585/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.2866
Epoch 586/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.2849
Epoch 587/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.2834
Epoch 588/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.2814
Epoch 589/800
32/32 [==============================] - 0s 9ms/step - loss: 1288

32/32 [==============================] - 0s 9ms/step - loss: 1288.1394
Epoch 675/800
32/32 [==============================] - 0s 10ms/step - loss: 1288.1379
Epoch 676/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.1362
Epoch 677/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.1343
Epoch 678/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.1328
Epoch 679/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.1310
Epoch 680/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.1294
Epoch 681/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.1271
Epoch 682/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.1257
Epoch 683/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.1240
Epoch 684/800
32/32 [==============================] - 0s 9ms/step - loss: 1288.1223
Epoch 685/800
32/32 [==============================] - 0s 9ms/step - loss: 128

32/32 [==============================] - 0s 9ms/step - loss: 1287.9891
Epoch 771/800
32/32 [==============================] - 0s 9ms/step - loss: 1287.9873
Epoch 772/800
32/32 [==============================] - 0s 9ms/step - loss: 1287.9860
Epoch 773/800
32/32 [==============================] - 0s 9ms/step - loss: 1287.9847
Epoch 774/800
32/32 [==============================] - 0s 9ms/step - loss: 1287.9829
Epoch 775/800
32/32 [==============================] - 0s 9ms/step - loss: 1287.9811
Epoch 776/800
32/32 [==============================] - 0s 9ms/step - loss: 1287.9794
Epoch 777/800
32/32 [==============================] - 0s 9ms/step - loss: 1287.9777
Epoch 778/800
32/32 [==============================] - 0s 9ms/step - loss: 1287.9767
Epoch 779/800
32/32 [==============================] - 0s 9ms/step - loss: 1287.9747
Epoch 780/800
32/32 [==============================] - 0s 9ms/step - loss: 1287.9734
Epoch 781/800
32/32 [==============================] - 0s 9ms/step - loss: 1287

Epoch 1/800
32/32 [==============================] - 1s 15ms/step - loss: 698427.0000
Epoch 2/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2687
Epoch 3/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2682
Epoch 4/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 5/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2679
Epoch 6/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 7/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 8/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2683
Epoch 9/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2683
Epoch 10/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2687
Epoch 11/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 12/800
32/32 [==============================] - 0s 15ms/step - los

32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 98/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2684
Epoch 99/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2682
Epoch 100/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2684
Epoch 101/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2683
Epoch 102/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2683
Epoch 103/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 104/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 105/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 106/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2682
Epoch 107/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 108/800
32/32 [==============================] - 0s 14ms/step - 

32/32 [==============================] - 0s 14ms/step - loss: 1289.2684
Epoch 193/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 194/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 195/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2686
Epoch 196/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2684
Epoch 197/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2683
Epoch 198/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 199/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 200/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 201/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2681
Epoch 202/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 203/800
32/32 [==============================] - 0s 14ms/step 

32/32 [==============================] - 0s 15ms/step - loss: 1289.2681
Epoch 288/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2683
Epoch 289/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 290/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 291/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2682
Epoch 292/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2679
Epoch 293/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 294/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 295/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2684
Epoch 296/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 297/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 298/800
32/32 [==============================] - 0s 14ms/step 

32/32 [==============================] - 0s 15ms/step - loss: 1289.2682
Epoch 383/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 384/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 385/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 386/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 387/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 388/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2679
Epoch 389/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 390/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 391/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 392/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 393/800
32/32 [==============================] - 0s 14ms/step 

32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 478/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 479/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 480/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 481/800
32/32 [==============================] - 1s 16ms/step - loss: 1289.2679
Epoch 482/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2679
Epoch 483/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2681
Epoch 484/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2683
Epoch 485/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 486/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2678
Epoch 487/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 488/800
32/32 [==============================] - 0s 15ms/step 

32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 573/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 574/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2679
Epoch 575/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 576/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 577/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 578/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 579/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 580/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 581/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2679
Epoch 582/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2679
Epoch 583/800
32/32 [==============================] - 0s 15ms/step 

32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 668/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2678
Epoch 669/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2681
Epoch 670/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 671/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 672/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 673/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2679
Epoch 674/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 675/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 676/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2683
Epoch 677/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 678/800
32/32 [==============================] - 0s 14ms/step 

32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 763/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2684
Epoch 764/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2686
Epoch 765/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 766/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 767/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 768/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 769/800
32/32 [==============================] - 0s 15ms/step - loss: 1289.2686
Epoch 770/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 771/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2682
Epoch 772/800
32/32 [==============================] - 0s 14ms/step - loss: 1289.2681
Epoch 773/800
32/32 [==============================] - 0s 14ms/step 

In [8]:
model.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=100)

Epoch 1/100


2023-06-06 13:43:42.830239: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 13:43:42.832300: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 13:43:42.833851: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

32/32 [==============================] - 3s 14ms/step - loss: 1289.2213
Epoch 2/100
32/32 [==============================] - 0s 14ms/step - loss: 1288.9812
Epoch 3/100
32/32 [==============================] - 0s 13ms/step - loss: 1288.8384
Epoch 4/100
32/32 [==============================] - 0s 14ms/step - loss: 1288.6508
Epoch 5/100
32/32 [==============================] - 0s 14ms/step - loss: 1288.4995
Epoch 6/100
32/32 [==============================] - 0s 13ms/step - loss: 1288.3353
Epoch 7/100
32/32 [==============================] - 0s 13ms/step - loss: 1288.3080
Epoch 8/100
32/32 [==============================] - 0s 13ms/step - loss: 1288.1077
Epoch 9/100
32/32 [==============================] - 0s 14ms/step - loss: 1287.9196
Epoch 10/100
32/32 [==============================] - 0s 13ms/step - loss: 1287.7749
Epoch 11/100
32/32 [==============================] - 0s 14ms/step - loss: 1287.6252
Epoch 12/100
32/32 [==============================] - 0s 14ms/step - loss: 1287.4792
E

32/32 [==============================] - 0s 14ms/step - loss: 1285.6792
Epoch 98/100
32/32 [==============================] - 0s 14ms/step - loss: 1285.8336
Epoch 99/100
32/32 [==============================] - 0s 13ms/step - loss: 1285.6500
Epoch 100/100
32/32 [==============================] - 0s 14ms/step - loss: 1285.6492


In [16]:
from sklearn.metrics import mean_squared_error as mse
Y_predicted = model.predict(np.array(X_test))
Y_ann_predicted = model_ann.predict(np.array(X_test))
Y_conv_predicted = model_conv.predict(np.array(X_test))
print(mse(Y_predicted, Y_test_flat))
print(mse(Y_ann_predicted, Y_test_flat))

11/11 [==============================] - 0s 3ms/step
124.40582499671201
124.08431316443712


In [ ]:
Y_predicted = [np.reshape(i, (14,14)) for i in Y_predicted]
Y_predicted

In [ ]:
Y_ann_predicted = [np.reshape(i, (14,14)) for i in Y_ann_predicted]
Y_ann_predicted

In [ ]:
print(Y_conv_predicted)

# AutoKeras

In [17]:
import autokeras as ak
import numpy as np

In [22]:
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
train_set = tf.data.Dataset.from_tensor_slices((X_train, Y_train_flat))
test_set = tf.data.Dataset.from_tensor_slices((X_test, Y_test_flat))


In [ ]:

input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=100
)
reg.fit(train_set,epochs=100, validation_split=0.15)
predicted_y = reg.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)|
print(reg.evaluate(test_set))


Trial 23 Complete [00h 00m 11s]
val_loss: 12058.470703125

Best val_loss So Far: 12008.6728515625
Total elapsed time: 00h 06m 04s

Search: Running Trial #24

Value             |Best Value So Far |Hyperparameter
False             |True              |structured_data_block_5/normalize
True              |True              |structured_data_block_5/dense_block_1/use_batchnorm
2                 |2                 |structured_data_block_5/dense_block_1/num_layers
512               |512               |structured_data_block_5/dense_block_1/units_0
0.5               |0.5               |structured_data_block_5/dense_block_1/dropout
128               |128               |structured_data_block_5/dense_block_1/units_1
0                 |0                 |regression_head_5/dropout
adam              |adam              |optimizer
0.001             |0.001             |learning_rate
256               |None              |structured_data_block_5/dense_block_1/units_2



2023-06-06 14:17:34.149246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [998,196]
	 [[{{node Placeholder/_1}}]]
2023-06-06 14:17:34.149650: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_21' with dtype double and shape [998,128]
	 [[{{node Placeholder/_21}}]]
2023-06-06 14:17:34.250884: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype double and sha

In [ ]:
regressor = ak.StructuredDataRegressor(max_trials=100, overwrite=True)
regressor.fit(train_set, epochs=300, validation_split=0.15)
predicted_y = regressor.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
print(regressor.evaluate(test_set))

In [ ]:
model = reg.export_model()
model.summary()

In [ ]:
model_regressor = regressor.export_model()
model_regressor.summary()

In [ ]:
X_test = X_test.reshape(-1, 128)
Y_ak_predicted =reg.predict(X_test)

# Autogoal